# Общий код между ноутбуками

In [ ]:
from sklearn.model_selection import train_test_split
from IPython.display import clear_output 
from sklearn.base import clone

# Дискретизация

In [ ]:
def entropy(a):
    vc = a.value_counts()
    vc/=vc.sum()
    return -(vc*np.log(vc)).sum()

def inconfidence_score(y_true, y_pred):
    return np.mean((1-y_pred)*y_true + y_pred*(1-y_true))

def discretize(data, variables, icat, icont, contdiscstrategy="kmeans", n_bins=3,
               automatic_fix=True
               
               ):
    transformers_data = dict()
    cont_features = [variables[i] for i in icont]
    
    if icat is None: # в датасете только непрерывные фичи (и предиктор)
        pipeline = make_pipeline(
            KBinsDiscretizer(n_bins=n_bins, encode="ordinal", random_state=42, strategy=contdiscstrategy), 
            FunctionTransformer(lambda x: x.astype("int")))
    elif icont is None: # в датасете только категориальные фичи (и предиктор)
        pipeline = make_pipeline(
            OrdinalEncoder(categories="auto"), 
            FunctionTransformer(lambda x: x.astype("int")))
    else:
        pipeline = make_pipeline(make_union(
        make_pipeline(FunctionTransformer(lambda x: x.iloc[:, icat]), OrdinalEncoder(categories="auto")),
        make_pipeline(FunctionTransformer(lambda x: x.loc[:, icont]), KBinsDiscretizer(n_bins=n_bins, encode="ordinal", random_state=42, strategy=contdiscstrategy))
    ),
        FunctionTransformer(lambda x: x.astype("int")))
    
    
    pipeline.fit(data)
    encoded_data = pd.DataFrame(pipeline.transform(data), columns=data.columns if icat is None or icont is None\
                                                                                        else data.columns[icat+icont])
    encoded_data = encoded_data[variables] # в общем случае пайплайн переставляет признаки, возвращаем их на их места здесь

    warning_splits_features = []

    for feat in encoded_data.columns:
        if entropy(encoded_data[feat]) < 0.85 and feat in cont_features:
            warning_splits_features.append(feat)
            print(f"Warning: feature {feat} has practically degenerate states and low entropy")

    if automatic_fix:
        q = np.linspace(0, 1, 11)



        dicval = {feat: [data[feat].quantile(qu) for qu in q] for feat in cont_features}
        if icat is None:
            v = pipeline.steps[0][1].bin_edges_
            f = list(pipeline.steps[0][1].get_feature_names_out())
        elif icat is not None and icont is not None:
            v = pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].bin_edges_
            f = list(pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].get_feature_names_out())
        indexes_out = [i for i, feat in enumerate(f) if feat in warning_splits_features]


        for i in indexes_out:
            v[i] = [v[i][0], dicval[f[i]][3], dicval[f[i]][7], v[i][3]]
        
        if icat is None:
            pipeline.steps[0][1].bin_edges_ = v
        elif icat is not None and icont is not None:
            pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].bin_edges_ = v
        pipeline.steps[0][1].bin_edges_ = v
        encoded_data = pd.DataFrame(pipeline.transform(data), columns=data.columns if icat is None or icont is None\
                                                                                        else data.columns[icat+icont])

    transformers_data["transformer"] = pipeline
    
    if icat is None:
        transformers_data["cont_features"]       = list(pipeline.steps[0][1].get_feature_names_out())
        transformers_data["cont_features_edges"] = pipeline.steps[0][1].bin_edges_
        
    elif icont is None:
        transformers_data["cat_features"]            = list(pipeline.steps[0][1].get_feature_names_out())
        transformers_data["cat_features_categories"] = pipeline.steps[0][1].categories_
        
    else:
        transformers_data["cont_features"]           = list(pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].get_feature_names_out())
        transformers_data["cont_features_edges"]     = pipeline.steps[0][1].named_transformers["pipeline-2"].steps[1][1].bin_edges_
        transformers_data["cat_features"]            = list(pipeline.steps[0][1].named_transformers["pipeline-1"].steps[1][1].get_feature_names_out())
        transformers_data["cat_features_categories"] = pipeline.steps[0][1].named_transformers["pipeline-1"].steps[1][1].categories_
    
    #kmeanspipeline.steps[0][1].named_transformers["pipeline-1"].steps[1][1].categories_[0]
    return encoded_data, transformers_data

#disc_data, pipeline_data = discretize(data, variables, icat=None, icont=[0, 1])


## Сборка таблиц усл. вероятностей по БС с градациями

In [ ]:
def collect_all_cpds(bn_info, distributions, n_states_map): # moved to GIEBN
    cpds = list()
    for index, row in bn_info.iterrows():
        feat = row["name"].name
        if len(row["parents"]) == 0:
            # cpd is just a pd
            cpd = TabularCPD(feat, n_states_map[feat], [[e] for e in distributions[feat]["cprob"]])
            cpds.append(cpd)
        else:
            cpd_list = [probs for probs in distributions[feat]["cprob"].values()]
            #cpd_list = [probs for i, probs in distributions[feat]["cprob"].items() if i[0]!="["]
            #print(cpd_list)
            nrows = len(cpd_list)
            ncols = len(cpd_list[0])
            cpd_list = [[cpd_list[i][j] for i in range(nrows)] for j in range(ncols)]
            #print(feat, row["parents"])
            #print(cpd_list)
            cpd = TabularCPD(feat, n_states_map[feat], cpd_list, evidence=row["parents"], evidence_card=[n_states_map[p] for p in row["parents"]])
            cpds.append(cpd)
    return cpds

In [ ]:
def bootstrap_sampling(data,
                       d_dict,# словарь списков фактор-градация
                       b_sample_size,
                       metrics_list, trials=1000, alpha=0, incl_random_removal=False,
                       mode='regr',
                       drop_mode='random',
                       reference='bn',
                       incl_test=False,
                       test_size=0.3,
                       stratify_tts=True):
    
    n_ref=np.inf

    metrics_results={
        k: [list() for _ in range(len(metrics_list))] for k in d_dict.keys()
    }
    metrics_results['init'] = [list() for _ in range(len(metrics_list))]

    

    if incl_random_removal:
        metrics_results['random'] = [list() for _ in range(len(metrics_list))]

    metrics_results_test = {k: [list() for _ in range(len(metrics_list))] for k in metrics_results.keys()}

    n_dropped_stats={k: list() for k in metrics_results.keys()}

    #пробуем побутсрапировать выборку, чтобы оценить значимость различий в ошибках регрессии
    for i in tqdm(range(trials)):
        r_seed = 42+i
        np.random.seed(r_seed)
        #indexes = np.random.choice(data.index, size=b_sample_size)
        #bsample = data.loc[indexes].reset_index()[data.columns]
        if not incl_test:
            bsample = data.sample(n=b_sample_size, replace=False).reset_index()[data.columns]
        else:
            bsample_train, b_sample_test = train_test_split(data, random_state=r_seed,
                                                            test_size=test_size,
                                                            stratify=data[target] if stratify_tts else None)
            bsample = bsample_train.sample(n=b_sample_size, replace=True).reset_index()[data.columns]
        
        model_base = copy(model)
        model_base.fit(bsample[features], bsample[target])

        if mode=='regr':
            y_pred = model_base.predict(bsample[features]) # depends on task (regr/classif)
        else:
            y_pred = model_base.predict_proba(bsample[features])[:, 1] # depends on task (regr/classif)

        data_errors = bsample.copy(deep=True)
        if mode=='regr':
            data_errors["ape_error"] = np.abs((bsample[target]-y_pred)/bsample[target])
        else:
            #inconf scode
            data_errors["inconf_error"]=(1-y_pred)*bsample[target]+(y_pred)*(1-bsample[target])


        data_errors.drop(columns=[target], inplace=True)

        bsample_disc = pd.DataFrame(pipeline_data["transformer"].transform(data_errors), columns=data_errors.columns)
        
        bsample_disc = bsample_disc.reindex(index=data_errors.index)
        
        #print(pipeline_data["transformer"].transform(data_errors).shape[0])
        iterlist = [reference]+[x for x in metrics_results.keys() if x != reference]

        for g in iterlist:
            mask = False
            if g == 'init':
                mask = None
                samp = bsample
            elif g == 'random':
                mask=None
                indexes_to_drop = np.random.choice(bsample.index, size=n_ref, replace=False)
                samp = bsample.drop(index=indexes_to_drop)
            else:
                mask=False
                for feat, cat in zip(d_dict[g][0], d_dict[g][1]):
                    mask = mask | (bsample_disc[feat]==cat)
                
                if drop_mode == 'random':
                    indexes_to_drop = np.random.choice(bsample[mask].index, size=int(np.floor(bsample[mask].shape[0]*(1-alpha))), replace=False)
                elif drop_mode == 'metric':
                    if mode=='regr':
                        errors = data_errors[mask]['ape_error']
                    else:
                        #inconf scode
                        #print(g)
                        #print(mask.index, data_errors.index)
                        #print(bsample.shape[0], bsample_disc.shape[0], data_errors.shape[0], mask.shape, indexes.shape[0])
                        errors = data_errors[mask]["inconf_error"]

                    errors_sorted = errors.sort_values(ascending=False)
                    size = min(n_ref, int(np.floor(errors.shape[0]*(1-alpha))))

                    # т.к. подходы могут отбрасывать разное кол-во наблюдений, получаемый результат по метрикам будет разным из-за этого

                    #thresh = errors_sorted[:size].min()
                    #indexes_to_drop = errors[errors>=thresh].index
                    thresh = errors_sorted[-size:].max()
                    indexes_to_drop = errors[errors<thresh].index
                
                samp = bsample.drop(index=indexes_to_drop)
                if g==reference:
                    n_ref=indexes_to_drop.shape[0]
                    
                n_dropped_stats[g].append(indexes_to_drop.shape[0])
        # --
            
            X1, y1 = samp[features], samp[target]

            model1 = copy(model)
            model1.fit(X1, y1)  

            def calculate_metrics(model, X_data, y_data):
                dict_m = {}
                if mode=='regr':
                    y1_pred = model.predict(X_data) # depends on task (regr/classif)
                else:
                    y1_pred = model.predict_proba(X_data)[:, 1] # depends on task (regr/classif)

                for k, metric in enumerate(metrics_list):
                    dict_m[k] = metric(y_data, y1_pred)

                return dict_m



            train_res = calculate_metrics(model1, X1, y1)
            for k, metric in enumerate(metrics_list):
                metrics_results[g][k].append(train_res[k])

            if incl_test:
                test_res = calculate_metrics(model1, b_sample_test[features], b_sample_test[target])
                for k, metric in enumerate(metrics_list):
                    metrics_results_test[g][k].append(test_res[k])

    return_value = {'metrics': metrics_results,
             'n_dropped': n_dropped_stats,
             'avg_n_dropped': {k: sum(v)/len(v) if len(v) >0 else -1 for k, v in n_dropped_stats.items()}}
    
    if incl_test:
        return_value['metrics_test'] = metrics_results_test

    return return_value

In [ ]:
def extract_categories(data):
    return [f"{feat}_{int(k)}" for feat in data.columns for k in
            sorted(data[feat].unique())]

In [ ]:
def gbn_training_pipeline(gbn, disc_data_encoded, error_name, categories, score=BicScore):
    gbn.add_nodes({"types": {feat: "disc" for feat in disc_data_encoded.columns}})
    features_cats = [x for x in disc_data_encoded.columns if error_name not in x]
    error_cats = [x for x in disc_data_encoded.columns if error_name in x]

    blacklist = [(x, y) for x, y in product(error_cats, features_cats)] \
            + [(x, y) for x, y in product(categories, categories) if x.split("_")[:-1]==y.split("_")[:-1]]

    gbn.add_edges(disc_data_encoded, scoring_function=("K2", score), params={"bl_add": blacklist})
    gbn.fit_parameters(data=disc_data_encoded, n_jobs=1)
    

In [ ]:
def measure_error(data, model, features, target, error_col_name='inconf_error', task='classif'):
    data_errors = data.copy(deep=True)
    if task=='classif':
        probs = model.predict_proba(data[features])[:, 1]
        data_errors[error_col_name] = (1-probs)*data[target]+(probs)*(1-data[target]) #inconfidence measure: the larger the worser inconfidence
    

    elif task=='regr':
        data_errors[error_col_name] = np.abs((data[target]-model.predict(data[features]))/data[target])
    data_errors.drop(columns=[target], inplace=True)
    return data_errors

In [ ]:
def get_all_predecessors(gbn, nodes):
    res = []
    iter_list=nodes
    while len(iter_list)>0:
        node = iter_list[0]
        iter_list=iter_list[1:]
        if gbn[node] is not None:
            pred = gbn[node].disc_parents
            if pred is not None:
                for p in pred:
                    if p not in res and p not in nodes:
                        res.append(p)
                        iter_list.append(p)
    return res


def get_prob(gbn, node, nodes_values):
    if len(gbn[node].disc_parents)==0:
        return gbn.get_dist(node)['cprob'][int(nodes_values[node])]
    else:
        return gbn.get_dist(node, nodes_values)[int(nodes_values[node])]


def calculate_cond_prob_support(gbn, e_grad, x_grad):
    pgmpy_gbn = gbn.to_pgmpy()
    if x_grad in gbn.isolated_nodes or e_grad in gbn.isolated_nodes:
        return 0
    inference = VariableElimination(pgmpy_gbn)
    res_infer = inference.query([e_grad, x_grad])
    dict_to_get_value = {e_grad:1, x_grad:1}
    
    return res_infer.get_value(**dict_to_get_value)


def calculate_mutual_information(gbn, feat1, feat2, grad_map):


    joint_values = {(x, y): calculate_cond_prob_support(gbn, x, y)
                    for x, y in product(grad_map[feat1], grad_map[feat2])}
    
    all_sum = sum([x for x in joint_values.values()])

    if all_sum == 0: # all gradations of feats are independent
        return 0

    joint_values = {k: v/all_sum for k, v in joint_values.items()}
    marg_probs = {}
    for grad in grad_map[feat1]:
        s=sum([v for k, v in joint_values.items() if k[0]==grad])
        marg_probs[grad]=s

    for grad in grad_map[feat2]:
        s=sum([v for k, v in joint_values.items() if k[1]==grad])
        marg_probs[grad]=s

    mutual_info = sum([joint_values[(x, y)]*np.log(joint_values[(x, y)]/(marg_probs[x]*marg_probs[y]) if marg_probs[x]*marg_probs[y]*joint_values[(x, y)] > 0 else 0
                                                   ) for x, y in product(grad_map[feat1], grad_map[feat2])])

    return mutual_info

In [ ]:
def error_based_importance(data, error_col, feature, feat_to_grad):
    error_probs = {grad: data[grad].mean() for grad in data.columns if error_col in grad}
    s=0
    for grad in feat_to_grad[feature]:
        sub_sum = 0
        for e, p in error_probs.items():
            cprob = data[(data[e]==1)&(data[grad]==1)].shape[0]/data.shape[0]/data[grad].mean()
            sub_sum+=cprob*np.log(cprob/p) if cprob>0 else 0
        s+=data[grad].mean()*sub_sum
    return s

def mutual_information(data, error_col, feature, feat_to_grad):
    s=0
    for e_grad, f_grad in product([grad for grad in data.columns if error_col in grad], 
                                  feat_to_grad[feature]):
        joint_prob = data[(data[e_grad]==1)&(data[f_grad]==1)].shape[0]/data.shape[0]
        px, py = data[e_grad].mean(), data[f_grad].mean()

        s+=joint_prob*np.log(joint_prob/px/py) if joint_prob>0 else 0

    return s

def error_based_importance_features(data, error_col, features, feat_to_grad):
    df = pd.DataFrame(columns=['Feature', 'Metric value'])
    for i, feature in enumerate(features):
        val = error_based_importance(data, error_col, feature, feat_to_grad)
        df.loc[i, :] = [feature, val]
    return df

def error_based_importance_features_bn(gbn, error_col, features, feat_to_grad):
    df = pd.DataFrame(columns=['Feature', 'Metric value'])
    for i, feature in tqdm(enumerate(features), total=len(features)):
        val = calculate_mutual_information(gbn, error_col, feature, feat_to_grad)
        df.loc[i, :] = [feature, val]
    return df

In [ ]:
def score_classif(model, X, y):
    y_pred = model.predict_proba(X)[:, 1]
    return m.roc_auc_score(y, y_pred)

def score_regr(model, X, y):
    y_pred = model.predict(X)
    return -m.mean_absolute_percentage_error(y, y_pred) #negative mape

def optunize_model(objective, features, target, data_dict, n_trials=30):
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction='maximize', sampler=sampler)  
    study.optimize(lambda trial: objective(trial, features, target, data_dict), n_trials=n_trials, show_progress_bar=True) #objective is implemented outside

    return study.best_params



In [ ]:
def single_run(model, data_dict, features_chosen, target, metric_dict, task='classif'):
    model.fit(data_dict['train'][features_chosen], data_dict['train'][target])

    if task == 'classif':
        y_pred_test = model.predict_proba(data_dict['test'][features_chosen])[:, 1]
    elif task == 'regr':
        y_pred_test = model.predict(data_dict['test'][features_chosen])
    metrics_vals = []
    metrics_chng=[]
    for m_name, metric in metric_dict.items():
        val = metric(data_dict['test'][target], y_pred_test)
        metrics_vals.append(np.round(val, 3))
    return metrics_vals

In [ ]:
def elimination_experiment(model, data_dict, metric_dict, importances, direction='asc', task='classif'):
    ordered_features = importances.sort_values(by='Metric value', ascending=(direction=='asc'))
    ordered_features = ordered_features['Feature'].tolist()
    df_res = pd.DataFrame(columns=['n_features', 'last_excluded']+[m_name for m_name in metric_dict.keys()]\
                                                                +[m_name+'_change, %' for m_name in metric_dict.keys()])
    for i in tqdm(range(len(ordered_features))):
        model_c = clone(model)
        features_chosen = ordered_features[i:]
        # here we should choose new optimal parameters
        
        #model_c = model._class_

        model_c.fit(data_dict['train'][features_chosen], data_dict['train'][target])

        if task == 'classif':
            y_pred_test = model_c.predict_proba(data_dict['test'][features_chosen])[:, 1]
        elif task == 'regr':
            y_pred_test = model_c.predict(data_dict['test'][features_chosen])
        metrics_vals = []
        metrics_chng=[]
        for m_name, metric in metric_dict.items():
            val = metric(data_dict['test'][target], y_pred_test)
            metrics_vals.append(np.round(val, 3))
            if i == 0:
                metrics_chng.append(0)
            else:
                base = df_res.loc[0, m_name]
                metrics_chng.append(np.round((val-base)/base*100, 3))
        df_res.loc[i, :] = [len(features_chosen), ordered_features[i-1] if i > 0 else '-']+metrics_vals+metrics_chng
    return df_res

In [ ]:
def elimination_experiment_v2(model, data_dict, metric_dict, importances, objective, target, direction='asc', task='classif', n_trials=20):
    ordered_features = importances.sort_values(by='Metric value', ascending=(direction=='asc'))
    ordered_features = ordered_features['Feature'].tolist()
    df_res = pd.DataFrame(columns=['n_features', 'last_excluded']+[m_name for m_name in metric_dict.keys()]\
                                                                +[m_name+'_change, %' for m_name in metric_dict.keys()])
    for i in tqdm(range(len(ordered_features))):
        model_c = clone(model)
        features_chosen = ordered_features[i:]
        # here we should choose new optimal parameters
        opt_parameters = optunize_model(objective, features_chosen, target, data_dict, n_trials)
        model_c = model.__class__(random_state=42, **opt_parameters)

        model_c.fit(data_dict['train'][features_chosen], data_dict['train'][target])
        if task == 'classif':
            y_pred_test = model_c.predict_proba(data_dict['test'][features_chosen])[:, 1]
        elif task == 'regr':
            y_pred_test = model_c.predict(data_dict['test'][features_chosen])
        

        metrics_vals = []
        metrics_chng=[]
        for m_name, metric in metric_dict.items():
            val = metric(data_dict['test'][target], y_pred_test)
            metrics_vals.append(np.round(val, 3))
            if i == 0:
                metrics_chng.append(0)
            else:
                base = df_res.loc[0, m_name]
                metrics_chng.append(np.round((val-base)/base*100, 3))
        df_res.loc[i, :] = [len(features_chosen), ordered_features[i-1] if i > 0 else '-']+metrics_vals+metrics_chng

        clear_output()
    return df_res

In [ ]:
def add_extra_features(data, features_orig,extra_size=0.5, mode='random', random_seed=42):
    extra_cnt = round(len(features_orig)/(1-extra_size))-len(features_orig)
    data_c = data.copy(deep=True)
    np.random.seed(random_seed)
    if mode=='random':
        for i in range(extra_cnt):
            std = np.random.uniform(0.75, 1.25)
            feat_name = f'extra_{str(i+1).zfill(3)}'
            data_c[feat_name] = np.random.normal(scale=std, size=data_c.shape[0])
    elif mode=='noisy':
        for i in range(extra_cnt):
            feature_chosen = np.random.choice(features_orig, size=1)[0]
            std = data[feature_chosen].std()
            feat_name = f'extra_{str(i+1).zfill(3)}'
            data_c[feat_name] = 0.5*data[feature_chosen]+0.5*np.random.normal(scale=std, size=data_c.shape[0])
    elif mode=='prod':
        first_subset=np.random.choice(features_orig, size=extra_cnt)
        second_subset=np.random.choice(features_orig, size=extra_cnt)
        i=0
        for feat1, feat2 in zip(first_subset, second_subset):
            feat_name = f'extra_{str(i+1).zfill(3)}'
            data_c[feat_name] = data[feat1]*data[feat2]
            i+=1
    return data_c

In [1]:
def get_best_record(df, col='ROC AUC', mode='max'):
    return df[df[col]==df[col].max()] if mode == 'max' else df[df[col]==df[col].min()]